In [1]:
%%capture
!pip install tensorboardX simpletransformers sklearn

In [2]:
!cat /usr/local/cuda/version.txt
!pip install pydash
!pip install torch==1.6.0
#conda install pytorch==1.1.0 torchvision==0.3.0 cudatoolkit=10.0 -c pytorch
#https://pytorch.org/get-started/previous-versions/

CUDA Version 10.0.130
     |████████████████████████████████| 748.8 MB 6.0 kB/s              


In [3]:
!rm -rf outputs

In [45]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import sklearn
from tqdm import tqdm
import pandas as pd

df = pd.read_csv("downloads/train_june20_all.csv")
len(df)

44632

In [49]:
#downsample medical data in training set
df = df[df["source"].isin(["oct3_labels", "june152022_labels_matscichem", 
                           "june152022_labels_envgeosoc", "labels_oct7", 
                           "psych1k_100522", "biology1k_050522", "predicts_acl_pm2500_vocabed", 
                           'pm_CONCLUSIONS', 'biology1k_050522'])]

In [50]:
df= df[["text", "labels"]]

In [53]:
df =df.dropna()
df = df.rename(columns={"labels": "label"})

In [54]:
df= df[["text", "label"]]
df.label = df.label.map(int)

In [55]:
df

,text,label
996,"While often idiopathic, anterior ischemic opti...",0
997,Primary cutaneous B-cell lymphomas (PCBCLs) ar...,0
998,The insufflation of gas during colonoscopy lea...,0
999,The aim of the present study is to investigate...,0
1000,The aim of this study was to assess predictive...,0
...,...,...
44627,Approximately 50% of patients with schizophren...,1
44628,Isoconversion processing of thermoanalytical d...,1
44629,The lack of social attributes in online teachi...,1
44630,Subsurface inspection of food and agricultural...,1


In [56]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

df2 = df.sample(frac=1)
# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. 
#The first column is the text with type str, and the second column is the label with type int.
size = len(df2)
train_cutoff = round(size*0.8)
train_df= df2[:train_cutoff]
eval_df = df2[train_cutoff:]

model_args = ClassificationArgs(save_steps=-1, save_model_every_epoch=False, sliding_window=True)

model = ClassificationModel('distilbert', 'distilbert-base-uncased',args=model_args, use_cuda=True) # You can set class weights by using the optional weight argument

INFO:filelock:Lock 139657795378704 acquired on /home/ec2-user/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333.lock


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

INFO:filelock:Lock 139657795378704 released on /home/ec2-user/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333.lock
INFO:filelock:Lock 139657795323496 acquired on /home/ec2-user/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock


Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

INFO:filelock:Lock 139657795323496 released on /home/ec2-user/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCla

...

INFO:filelock:Lock 139657795379096 released on /home/ec2-user/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
INFO:filelock:Lock 139657795325344 acquired on /home/ec2-user/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

INFO:filelock:Lock 139657795325344 released on /home/ec2-user/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
INFO:filelock:Lock 139657795379096 acquired on /home/ec2-user/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

INFO:filelock:Lock 139657795379096 released on /home/ec2-user/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


In [57]:
# Train the model
model.train_model(train_df, overwrite_output_dir=True)
# Evaluate the model, u
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/7982 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 7993 features created from 7982 samples.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of distilbert model complete. Saved to outputs/.
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:1443: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


  0%|          | 0/1996 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 1996 features created from 1996 samples.


Running Evaluation:   0%|          | 0/250 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.7803179340475614, 'tp': 548, 'tn': 1254, 'fp': 130, 'fn': 64, 'acc': 0.9028056112224448, 'eval_loss': 0.2605627398118377}


In [58]:
result

{'mcc': 0.7803179340475614,
 'tp': 548,
 'tn': 1254,
 'fp': 130,
 'fn': 64,
 'acc': 0.9028056112224448,
 'eval_loss': 0.2605627398118377}

In [31]:
import pandas as pd

In [97]:
dfx = pd.read_csv("downloads/16M_small_alldomains_clean.csv",  lineterminator='\n', error_bad_lines=False)

In [98]:
dfx = dfx[dfx.fields.isin(['Environmental Science', 'Material Science', 'Psychology', 'Geology', 'Biology', 'Materials Science', 'Political Science'])]


In [99]:
dfx["text"] = dfx["text"].astype(str)



In [35]:
#dfx = dfx.sample(frac=0.2)

In [100]:
len(dfx)

4707344

In [82]:
dfx.fields.value_counts(normalize=True) * 100

Biology                  37.001077
Materials Science        28.016308
Psychology               15.471485
Political Science         7.556533
Geology                   6.193726
Environmental Science     5.760871
Name: fields, dtype: float64

In [101]:
dfx =dfx.groupby('fields').apply(lambda x: x.sample(10000))

In [102]:
dfx =dfx.groupby('fields').apply(lambda x: x.sample(10000))

In [63]:
#dfx.to_csv("downloads/sample_bioadjacent600k.csv", index = False, header=True)

In [84]:
predicts_path = 'downloads/predicts_20June_16M.txt'

In [103]:
def stepwise(step, df, f):
    l = []
    for idx in range(step, len(df)+step-1, step):
        print(idx-step, idx)# dataframe[idx-step:idx])
        sl = df[idx-step:idx]
        batch = model.predict(list(sl))
        l = l + list(batch[0])
        with open(predicts_path, 'w') as f:
            for item in l:
                f.write("%s\n" % item)
    return l

In [104]:
predicts = stepwise(20000, dfx["text"], model.predict)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


0 20000


  0%|          | 0/20000 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: 20000 features created from 20000 samples.


  0%|          | 0/2508 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.
INFO:simpletransformers.classification.classification_model: Sliding window enabled


40000 60000


  0%|          | 0/20000 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (661 > 512). Running this sequence through the model will result in indexing errors
INFO:simpletransformers.classification.classification_model: 20000 features created from 20000 samples.


  0%|          | 0/2524 [00:00<?, ?it/s]

In [105]:
with open(predicts_path, 'r') as f:
    preds = f.read().split()#f.write("%s\n" % item)
                

In [106]:
len(preds)

60000

In [107]:
#dfx = dfx[:3000000]
dfx["predicts"] = preds

In [108]:
dfx["predicts"] = dfx["predicts"].map(int)

In [109]:
dfx = dfx[dfx["predicts"]==1]
dfx = dfx[dfx["text"] != "nan"]

8410

In [110]:
len(df)

9978

In [113]:
dfx.fields.value_counts()

Environmental Science    2712
Biology                  1662
Psychology               1253
Geology                  1173
Materials Science         834
Political Science         776
Name: fields, dtype: int64

In [115]:
len(dfx)

8410

In [116]:
dfx=dfx[dfx["doi"].notna()]

In [118]:
dfo = pd.read_csv("downloads/train_june20_all.csv")
dd = dfx[~dfx["doi"].isin(list(dfo["doi"]))]
dfx=dd

In [120]:
dfx["fields"].value_counts()

Environmental Science    2615
Biology                  1632
Psychology               1229
Geology                  1135
Materials Science         830
Political Science         776
Name: fields, dtype: int64

In [128]:
dr= dfx.iloc[::-1]
dr.to_csv("downloads/June20_non_medicine_sorted_predicts.csv", index = False, header=True)

text  \
fields                                                                   
Psychology 1744429   Hookah smoking is a harmful practice which has...   
           8252263   Misbehaviour phenomenon in elementary school c...   
           14080842  Suicide is one of the leading causes of premat...   
           8149704   Less-skilled adult readers have been found to ...   
           4679051   Postsecondary learning environments often util...   
...                                                                ...   
Biology    1989197   Rhabdomyosarcoma (RMS) is the most common type...   
           15376790  Despite extensive research, the mechanism of i...   
           7064261   Stings by insects from the Hymenoptera order c...   
           10018890  Recent studies have shown that neonicotinoid i...   
           2195958   The lymphatic vasculature is essential for tis...   

                                                       doi      fields  \
fields                                                                   
Psychology 1744429           10.1080/14659891.2019.1664655  Psychology   
           8252263          10.5958/0976-5506.2018.01363.3  Psychology   
           14080842          10.1016/S0924-9338(13)76596-9  Psychology   
           8149704                10.1177/0741713620952348  Psychology   
           4679051                  10.19030/AJBE.V3I7.455  Psychology   
...                                                    ...         ...   
Biology    1989197           10.1158/1078-0432.CCR-10-0166     Biology   
           15376790  10.3760/cma.j.issn.0366-6999.20112856     Biology   
           7064261               10.1016/j.pep.2010.03.029     Biology   
           10018890                    10.1038/nature13531     Biology   
           2195958           10.1158/0008-5472.CAN-08-1879     Biology   

                     predicts  
fields                         
Psychology 1744429          1  
           8252263          1  
           14080842         1  
           8149704          1  
           4679051          1  
...                       ...  
Biology    1989197          1  
           15376790         1  
           7064261          1  
           10018890         1  
           2195958          1  

[8217 rows x 4 columns]

In [97]:
df2 = dfx[dfx["fields"].isin(["Sociology", "Geology", "Environmental Science"])]

In [101]:
df2.to_csv("downloads/underrepresented_domains_predicts.csv", index = False, header=True)

In [102]:
df2 = dfx[dfx["fields"].isin(["Materials Science", "Psychology", "Chemistry"])]

In [105]:
df2.to_csv ("downloads/mid_represented_domains_predicts.csv", index = False, header=True)
#check that not in final110k_1605

In [ ]:
dfx.to_csv("downloads/pos_preds_12062022.csv", index = False, header=True)

In [ ]:
dfx[dfx["fieldsOfStudy"].isin(['Environmental Science', 'Sociology', 'Geology', 'Biology', 'Materials Science', 'Chemistry'])]

In [58]:
dfx

,index,text,s2Url,doi,journalName,title,fieldsOfStudy,fields,predicts
4,7,Falsified and substandard drugs are a global h...,https://semanticscholar.org/paper/0d0467c4cccb...,10.1371/journal.pone.0090601,PLoS ONE,Technologies for Detecting Falsified and Subst...,['Medicine'],Medicine,1
7,13,IntroductionMcCune-Albright syndrome is a comp...,https://semanticscholar.org/paper/a28cf4e1bf12...,10.1186/1752-1947-6-32,Journal of Medical Case Reports,Pamidronic acid and cabergoline as effective l...,['Medicine'],Medicine,1
72,147,"As men grow older, testosterone (T) levels dec...",https://semanticscholar.org/paper/6616f4ba272f...,10.4103/1008-682X.122357,Asian Journal of Andrology,Sex steroids and cardiovascular disease,['Medicine'],Medicine,1
103,203,Viral infections have the potential to induce ...,https://semanticscholar.org/paper/15bdef5b0f93...,10.1007/s11901-016-0292-z,Current Hepatology Reports,HCC: Where Does HCV Therapy Play a Role?,['Medicine'],Medicine,1
123,243,Mastocytosis is a myeloid neoplasm characteriz...,https://semanticscholar.org/paper/ec9263657cf8...,10.1586/ehm.10.42,Expert Review of Hematology,"Pathogenesis, classification and treatment of ...",['Medicine'],Medicine,1
...,...,...,...,...,...,...,...,...,...
5076747,4772128,The association of a benign ovarian tumor with...,https://semanticscholar.org/paper/b25d3eec1c5c...,10.4314/WAJM.V26I3.28322,West African journal of medicine,Meigs' syndrome presenting as haemorrhagic ple...,['Medicine'],Medicine,1
5076802,4772221,The putative effects of diabetes and metabolic...,https://semanticscholar.org/paper/164e57704ccb...,10.1210/JCEM.80.4.7536202,The Journal of clinical endocrinology and meta...,Insulin-like growth factor (IGF)-I and -II and...,['Medicine'],Medicine,1
5076812,4772243,Factor V deficiency secondary to inhibitors is...,https://semanticscholar.org/paper/ca68ab337e14...,10.1155/2015/603402,Case Reports in Hematology,Refractory Epistaxis due to Severe Factor V De...,['Medicine'],Medicine,1
5076833,4772289,Recent prostate cancer screening trials have g...,https://semanticscholar.org/paper/368327b75301...,10.1101/402743,bioRxiv,A natural history model for planning prostate ...,['Medicine'],Medicine,1


In [181]:
dt = pd.read_csv("core_pos.csv")
#d[d["first_sent"].str.contains(re.escape("|".join(d["authors"][2:-2].replace("'", "").split(","))).strip().replace('\\', ''))]
def formatStr (x):
    x = eval(x)
    x = [re.escape(v) for v in x]
    return "|".join(x)

regexs = d.authors.apply(formatStr)
matches = pd.Series()
for i, r in tqdm(d.iterrows()):
    matches.loc[i] = len(re.findall(regexs.loc[i], d.first_sent.loc[i])) > 0
dc["authors"] = dc["authors"].map(eval)
dc = dc.drop(columns=["fullText"])
dc.to_csv("core_pos_parsed.csv", index = False, header=True)
tt.to_csv("core_pos_min.csv", index = False, header=True)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
44962it [00:59, 760.29it/s] 


distilbert
'mcc': 0.7621432780395265, 'tp': 167, 'tn': 480, 'fp': 39, 'fn': 30, 'acc': 0.9036312849162011, 'eval_loss': 0.24669100075132316}

